#### NOT USING!!!!!!!! Makefiles for GRN inference

In [1]:
%%bash
code_dir="/ocean/projects/cis240075p/asachan/bio_informatics_analysis/multiome_B_Cell_analysis/B_Cell_analysis_code/method_runs/dictys_runs"
rm -Rf $code_dir/makefiles
mkdir $code_dir/makefiles
cd $code_dir/makefiles
dictys_helper makefile_template.sh common.mk config.mk env_none.mk static.mk


In [2]:
%%bash
code_dir="/ocean/projects/cis240075p/asachan/bio_informatics_analysis/multiome_B_Cell_analysis/B_Cell_analysis_code/method_runs/dictys_runs"
# Update configurations, such as:
# DEVICE: pytorch device, e.g. cpu, cuda:0. If you do not have a GPU, use 'cpu' and expect LONG computing time.
# GENOME_MACS2: effective genome size for macs2. See https://deeptools.readthedocs.io/en/develop/content/feature/effectiveGenomeSize.html
# JOINT: whether dataset is joint profiling of RNA and ATAC.
# Other configurations include quality control thresholds, number of threads in each job, number of hidden confounders, etc.
# They can be obtained in the full-multiome tutorial.
# NTH=4
dictys_helper makefile_update.py $code_dir/makefiles/config.mk '{"ENVMODE": "none", "NTH": "4", "DEVICE": "cuda:0", "GENOME_MACS2": "hs", "JOINT": "1"}'


In [1]:
!cat /ocean/projects/cis240075p/asachan/bio_informatics_analysis/multiome_B_Cell_analysis/B_Cell_analysis_code/method_runs/dictys_runs/makefiles/config.mk

# Lingfei Wang, 2022. All rights reserved.
#This file contains parameters for whole run and individual steps to be edited for your dataset
#This file should be edited to configure the run
#This file should NOT be directly used for any run with `makefile -f` 

############################################################
# Run environment settings
############################################################
#Which environment to use, corresponding to env_$(ENVMODE).mk file
ENVMODE=none
#Maximum number of CPU threads for each job
#This is only nominated and passed through to other softwares without any guarantee.
NTH=4
#Device name for pyro/pytorch
#Note: cuda devices other than cuda:0 could be incompatible with singularity environment
DEVICE=cuda:0

############################################################
# Dataset settings
############################################################

#Genome size for Macs2, accept shortcuts like mm & hs
GENOME_MACS2=hs
#Whether dataset is joint prof

In [20]:
%%bash
#run checks on dataset and makefiles
data="/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/data"
makefiles="/ocean/projects/cis240075p/asachan/bio_informatics_analysis/multiome_B_Cell_analysis/B_Cell_analysis_code/method_runs/dictys_runs/makefiles"

dictys_helper makefile_check.py --dir_data $data --dir_makefiles $makefiles

Joint profile: True
Found 35420 cells with RNA profile
Found 24036 genes with RNA profile
Found 35420 cells with ATAC profile
Found 769 motifs
Found 678 TFs
Found 461 TFs in current dataset
Missing 217 TFs in current dataset: ANDR,AP2A,AP2B,AP2C,AP2D,ARI3A,ARI5B,ATF6A,BARH1,BARH2,BC11A,BHA15,BHE22,BHE23,BHE40,BHE41,BMAL1,BRAC,BSH,COE1,COT1,COT2,CR3L1,CR3L2,ERR1,ERR2,ERR3,EVI1,GCR,HEN1,HMBX1,HME1,HME2,HNF6,HTF4,HXA1,HXA10,HXA11,HXA13,HXA2,HXA5,HXA7,HXA9,HXB1,HXB13,HXB2,HXB3,HXB4,HXB6,HXB7,HXB8,HXC10,HXC11,HXC12,HXC13,HXC6,HXC8,HXC9,HXD10,HXD11,HXD12,HXD13,HXD3,HXD4,HXD8,HXD9,ITF2,KAISO,MCR,MGAP,MLXPL,MYBA,MYBB,NDF1,NDF2,NF2L1,NF2L2,NFAC1,NFAC2,NFAC3,NFAC4,NGN2,NKX21,NKX22,NKX23,NKX25,NKX28,NKX31,NKX32,NKX61,NKX62,ONEC2,ONEC3,OZF,P53,P5F1B,P63,P73,PEBB,PHX2A,PHX2B,PIT1,PKNX1,PLAL1,PO2F1,PO2F2,PO2F3,PO3F1,PO3F2,PO3F3,PO3F4,PO4F1,PO4F2,PO4F3,PO5F1,PO6F1,PO6F2,PRD14,PRGR,RHXF1,RORG,RX,SMCA1,SMCA5,SRBP1,SRBP2,STA5A,STA5B,STF1,SUH,TF2LX,TF65,TF7L1,TF7L2,TFE2,THA,THA11,THB,TWST1,TYY1,TYY2,UBIP

#### debugging cell-matching

In [3]:
%%bash
psc_path="/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/data/expression.tsv.gz"
LOCAL_PATH="/dev/shm"

# Transfer with progress and verbose output
scp -v -C $psc_path $LOCAL_PATH

Executing: cp -- /ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/data/expression.tsv.gz /dev/shm


In [7]:
import pandas as pd
# load the expression matrix
expression_mtx = pd.read_csv('/dev/shm/expression.tsv', sep='\t', index_col=0)

In [8]:
#print the number of cells and genes
expression_mtx.shape

(24036, 35420)

In [9]:
# inspect the expression matrix
expression_mtx.head()

,AAACAGCCAAACCTTG-1,AAACAGCCAAAGCTAA-1,AAACAGCCAAGCCACT-1,AAACAGCCAAGGTGCA-1,AAACAGCCAAGTTATC-1,AAACAGCCAATAGCCC-1,AAACAGCCAATTATGC-1,AAACAGCCAGTTAGCC-1,AAACAGCCATAATCCG-1,AAACAGCCATTCAGCA-1,...,TTTGTTGGTGTTGCAA-1,TTTGTTGGTTAAGGTT-1,TTTGTTGGTTAGCGTA-1,TTTGTTGGTTATCCGT-1,TTTGTTGGTTGACTTC-1,TTTGTTGGTTTACGTC-1,TTTGTTGGTTTAGTCC-1,TTTGTTGGTTTATGGG-1,TTTGTTGGTTTCCTCC-1,TTTGTTGGTTTGAGGC-1
A1BG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
A1BG-AS1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M-AS1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Infer static GRNs

#### set makefile input/output directories right
steps in full tutorial are - 
1. make static.mk runs a bunch of i/o and scripts in this order: 
    * python3 -m dictys preproc selects_rna + preproc qc_reads + preproc selects_atac {combines cell bams to cell-type bams}
    * python3 -m dictys chromatin macs2 {find peaks from atac data using macs2}
    * python3 -m dictys chromatin wellington {TF footprinting with wellington and pyDNASE}
    * python3 -m dictys chromatin homer {scan the detected regions (peaks/footprints both) for TF motifs using homer}
    * python3 -m dictys chromatin binding {get binding score of the TF after region detection and scanning}
    * python3 -m dictys chromatin tssdist {TSS and nearby 1-d distance filtering of regions for getting more proabable regulatory bins - }
    * Get GRN TF-gene links:
        python3 -m dictys chromatin linking {(TF --- motif > scanning peaks and getting binding score > try to add alphafold binding predictions too) --- region (1-D proximity peak-peak correlation might give the promoters) --- gene > gtf} can use HiC info too at this step to get 3D peak-peak correlations by using their contact probability as pass the hic found genomic regions as whitelist.bed !!!
        python3 -m dictys chromatin binlinking {constraining the TF-gene links using ranking and then thresholding for top n ranked TFs or genes}

2. Re-run static.mk to reconstruct the GRN with refined edges from RNA-seq data
    * python3 -m dictys network reconstruct -h {use pytorch to infer the GRN with stochastic process model under the TF binding network constraint}
    * python3 -m dictys network normalize -h {normalize network edges based on the standard deviation of regulator and target genes. This can overcome biases in the estimation of variance of gene expression due to single-cell sparsity}
    * python3 -m dictys network indirect -h {}

****** python3 -m dictys network indirect -h
>>>>>> Computes steady-state indirect effect of gene perturbation from OU process.
Performs extra regularization on network by bounding the eigenvalues of
feedback loops with parameters eigmin and eigmax. Values away from 1 indicates
stronger feedback loop effects. Set values closer to 1 to apply stronger
regularization.
The total effect network is **not yet integrated into network analysis functions**. (can't be plotted or analysed, but network can be constructed for modeling TF perturbations !!!!)

In [2]:
# Removes CPU usage limit by some jupyter versions
import os
os.environ['KMP_AFFINITY'] = ''


In [3]:
# Define the data and code dirs
import os
import shutil

code_dir="/ocean/projects/cis240075p/asachan/bio_informatics_analysis/multiome_B_Cell_analysis/B_Cell_analysis_code/method_runs/dictys_runs"

DIRI="/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/data"        # input data directory
DIRO="/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs"     # output data directory
DIRT="/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static"    # temporary input directory 
DIRTI = DIRT
DIRTO = DIRT
FILE_SUBSET = os.path.join(DIRI, "subsets.txt")  # File for cell subsets
TMPDIR = "/local"       # Temporary folder

# Load the subset names from subsets.txt
with open(FILE_SUBSET, 'r') as file:
    subsets = [line.strip() for line in file if line.strip()]

# Ensure the output directory exists
os.makedirs(DIRTO, exist_ok=True)

# JOINT variable as per config.mk (assumed value here)
JOINT = 1  # Change this value according to your configuration

In [4]:
# Process each subset
for subset in subsets:
    # Define paths for names_rna.txt and names_atac0.txt
    names_rna_src = os.path.join(DIRI, "subsets", subset, "names_rna.txt")
    names_rna_dest = os.path.join(DIRTO, subset, "names_rna.txt")

    # Ensure destination directory exists
    os.makedirs(os.path.dirname(names_rna_dest), exist_ok=True)

    # Copy names_rna.txt
    shutil.copy(names_rna_src, names_rna_dest)

    if JOINT == 1:
        # If JOINT is 1, copy names_rna.txt to names_atac0.txt in DIRTO
        names_atac0_dest = os.path.join(DIRTO, subset, "names_atac0.txt")
        shutil.copy(names_rna_dest, names_atac0_dest)
    else:
        # If JOINT is not 1, copy names_atac.txt from DIRI/subsets to DIRTO
        names_atac_src = os.path.join(DIRI, "subsets", subset, "names_atac.txt")
        names_atac_dest = os.path.join(DIRTO, subset, "names_atac0.txt")

        # Ensure destination directory exists
        os.makedirs(os.path.dirname(names_atac_dest), exist_ok=True)

        shutil.copy(names_atac_src, names_atac_dest)

print("All subsets processed successfully.")

All subsets processed successfully.


## Getting TF-gene links from chromatin priors

#### CPU : preprocess select_rna: subsets the RNA data separately for cells of each context/cell type.

In [5]:
import os
import subprocess
from concurrent.futures import ProcessPoolExecutor

# Function to run the command for a single subset
def process_subset(subset):
    try:
        # Define paths
        names_rna_path = os.path.join(DIRTI, subset, "names_rna.txt")
        expression0_path = os.path.join(DIRTI, subset, "expression0.tsv.gz")

        # Build the command
        command = [
            "python3", "-m", "dictys", "preproc", "selects_rna",
            os.path.join(DIRI, "expression.tsv.gz"),
            names_rna_path,
            expression0_path
        ]

        # Run the command
        print(f"Running command for subset {subset}: {' '.join(command)}")
        subprocess.run(command, check=True)
        print(f"Finished processing subset {subset}")
    except subprocess.CalledProcessError as e:
        print(f"Error processing subset {subset}: {e}")
    except Exception as e:
        print(f"Unexpected error processing subset {subset}: {e}")



In [6]:
# Number of workers (cores) to use
num_workers = 128

# Use ProcessPoolExecutor to run multiple jobs in parallel
with ProcessPoolExecutor(max_workers=num_workers) as executor:
    # Map the subsets to the process_subset function
    executor.map(process_subset, subsets)

print("All subsets processed successfully.")

Running command for subset Subset0: python3 -m dictys preproc selects_rna /ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/data/expression.tsv.gz /ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static/Subset0/names_rna.txt /ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static/Subset0/expression0.tsv.gzRunning command for subset Subset2: python3 -m dictys preproc selects_rna /ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/data/expression.tsv.gz /ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static/Subset2/names_rna.txt /ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static/Subset2/expression0.tsv.gzRunning command for subset Subset1: python3 -m dictys preproc selects_rna /ocean/projects/cis240075p/asachan/datas

#### CPU : preprocess qc_reads : removes low-read genes and cells for cell-type specific quality control 

In [ ]:
################################################################## QC parameters to tune based on the dataset ########################################################
n_gene = 50 #min total read depth per gene
nc_gene = 10 #min number of cells expressed per gene
ncp_gene = 0 #min number of cells expressed per gene in percentage
n_cell = 200 #min total coverage per cell
nt_cell = 100 #min number of genes expressed per cell
ntp_cell = 0 #min number of genes expressed per cell in percentage

In [7]:
# Function to run the qc_reads command for a single subset
def process_qc_reads(subset):
    try:
        # Define paths
        expression0_path = os.path.join(DIRTI, subset, "expression0.tsv.gz")
        expression_path = os.path.join(DIRTI, subset, "expression.tsv.gz")

        # Build the command
        command = [
            "python3", "-m", "dictys", "preproc", "qc_reads",
            expression0_path,
            expression_path,
            str(n_gene), str(nc_gene), str(ncp_gene),
            str(n_cell), str(nt_cell), str(ntp_cell)
        ]

        # Run the command
        print(f"Running command for subset {subset}: {' '.join(command)}")
        subprocess.run(command, check=True)
        print(f"Finished processing subset {subset}")
    except subprocess.CalledProcessError as e:
        print(f"Error processing subset {subset}: {e}")
    except Exception as e:
        print(f"Unexpected error processing subset {subset}: {e}")

In [8]:
# Number of workers (cores) to use
num_workers = 128

# Use ProcessPoolExecutor to run multiple jobs in parallel
with ProcessPoolExecutor(max_workers=num_workers) as executor:
    # Map the subsets to the process_qc_reads function
    executor.map(process_qc_reads, subsets)

print("All subsets processed successfully.")

Running command for subset Subset0: python3 -m dictys preproc qc_reads /ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static/Subset0/expression0.tsv.gz /ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static/Subset0/expression.tsv.gz 50 10 0 200 100 0Running command for subset Subset5: python3 -m dictys preproc qc_reads /ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static/Subset5/expression0.tsv.gz /ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static/Subset5/expression.tsv.gz 50 10 0 200 100 0Running command for subset Subset2: python3 -m dictys preproc qc_reads /ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static/Subset2/expression0.tsv.gz /ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static/Su

Traceback (most recent call last):
  File "/ocean/projects/cis240075p/asachan/.conda/envs/dictys/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/ocean/projects/cis240075p/asachan/.conda/envs/dictys/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/ocean/projects/cis240075p/asachan/.conda/envs/dictys/lib/python3.9/site-packages/dictys/__main__.py", line 13, in <module>
Traceback (most recent call last):
  File "/ocean/projects/cis240075p/asachan/.conda/envs/dictys/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/ocean/projects/cis240075p/asachan/.conda/envs/dictys/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/ocean/projects/cis240075p/asachan/.conda/envs/dictys/lib/python3.9/site-packages/dictys/__main__.py", line 13, in <module>
Traceback (most recent call last):
  File "/ocean/projects

Error processing subset Subset7: Command '['python3', '-m', 'dictys', 'preproc', 'qc_reads', '/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static/Subset7/expression0.tsv.gz', '/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static/Subset7/expression.tsv.gz', '50', '10', '0', '200', '100', '0']' returned non-zero exit status 1.
Error processing subset Subset3: Command '['python3', '-m', 'dictys', 'preproc', 'qc_reads', '/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static/Subset3/expression0.tsv.gz', '/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/tmp_static/Subset3/expression.tsv.gz', '50', '10', '0', '200', '100', '0']' returned non-zero exit status 1.
Finished processing subset Subset4Error processing subset Subset8: Command '['python3', '-m', 'dictys', 'preproc', 'qc_reads', '/ocean/projects/

### Debugging the QC dropouts of cell-types

In [ ]:
## check for the failed subsets how the expression matrix is filtered. Compare with the passed subsets.